In [1]:
importall Base

In [2]:
include("frames.jl");

In [3]:
abstract CacheElement{T}

immutable ImmutableCacheElement{T} <: CacheElement{T}
    data::T
end

get{T}(element::ImmutableCacheElement{T}) = element.data
function setdirty!{T}(element::ImmutableCacheElement{T})
end

type MutableCacheElement{T} <: CacheElement{T}
    data::T
    updateFunction::Function
    dirty::Bool
    
    MutableCacheElement(data::T, updateFunction::Function) = new(data, updateFunction, true)
end
MutableCacheElement{T}(data::T, updateFunction::Function) = MutableCacheElement{T}(data, updateFunction)
MutableCacheElement(updateFunction::Function) = MutableCacheElement(updateFunction(), updateFunction)

function get{T}(element::MutableCacheElement{T})
    if element.dirty
        element.data = element.updateFunction()
        element.dirty = false
    end
    return element.data
end
function setdirty!{T}(element::MutableCacheElement{T})
    element.dirty = true
end

setdirty! (generic function with 2 methods)

In [4]:
type FrameCache{T}
    root::CartesianFrame3D
    transformsToParent::Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}
    transformsToRoot::Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}
    dependents::Dict{CacheElement{Transform3D{T}}, Vector{CacheElement{Transform3D{T}}}}
    
    function FrameCache(root::CartesianFrame3D)
        transformsToParent = Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}()
        toRoot = ImmutableCacheElement(Transform3D{T}(root, root))
        transformsToRoot = Dict{CartesianFrame3D, CacheElement{Transform3D{T}}}(root => toRoot)
        dependents = Dict{CacheElement{Transform3D{T}}, Vector{CacheElement{Transform3D{T}}}}(toRoot => [])
        new(root, transformsToParent, transformsToRoot, dependents)
    end
end

In [10]:
function setdirty!{T}(cache::FrameCache{T}, element::CacheElement{Transform3D{T}})
    setdirty!(element)
    for dependent in cache.dependents[element]
        setdirty!(cache, dependent)
    end
end

function setdirty!{T}(cache::FrameCache{T})
    for element in values(cache.transformsToParent)
        setdirty!(element)
    end
    for element in values(cache.transformsToRoot)
        setdirty!(element)
    end
end

function add_frame!{T}(cache::FrameCache{T}, t::Transform3D{T})
    # for fixed frames
    to_parent = ImmutableCacheElement(t)
    cache.dependents[to_parent] = []
    parent_to_world = cache.transformsToRoot[t.to]
    cache.transformsToParent[t.from] = to_parent

    updateTransformToWorld = () -> begin println("updating world 1"); get(parent_to_world) * get(to_parent) end
    to_world = MutableCacheElement(updateTransformToWorld)
    cache.dependents[to_world] = []
    push!(cache.dependents[parent_to_world], to_world)
    cache.transformsToRoot[t.from] = to_world

    setdirty!(cache)
    return cache
end

function add_frame!{T}(cache::FrameCache{T}, updateTransformToParent::Function, dependencies...)
    # for non-fixed frames
    to_parent = MutableCacheElement(() -> begin println("updating parent"); updateTransformToParent() end)
    cache.dependents[to_parent] = []
    for dependency in dependencies
        push!(cache.dependents[dependency], to_parent)
    end
    t = to_parent.data
    parent_to_world = cache.transformsToRoot[t.to]
    cache.transformsToParent[t.from] = to_parent
    
    updateTransformToWorld = () -> begin println("updating world 2"); get(parent_to_world) * get(to_parent) end
    to_world = MutableCacheElement(updateTransformToWorld)
    cache.dependents[to_world] = []
    push!(cache.dependents[to_parent], to_world)
    push!(cache.dependents[parent_to_world], to_world)
    cache.transformsToRoot[t.from] = to_world
    
    setdirty!(cache)
    return cache
end

function transform_to_parent{T}(cache::FrameCache{T}, frame::CartesianFrame3D)
    frame == cache.root && error("frame has no parent")
    return get(cache.transformsToParent[frame])
end

transform_to_world{T}(cache::FrameCache{T}, frame::CartesianFrame3D) = get(cache.transformsToRoot[frame])

function relative_transform{T}(cache::FrameCache{T}, from::CartesianFrame3D, to::CartesianFrame3D)
    return inv(transform_to_world(cache, to)) * transform_to_world(cache, from)
end

relative_transform (generic function with 1 method)

In [11]:
root = CartesianFrame3D("root")
cache = FrameCache{Float64}(root)
f1 = CartesianFrame3D("f1")
t1 = Transform3D(f1, root, nquatrand(), rand(Vec{3, Float64}))
add_frame!(cache, t1)
f2 = CartesianFrame3D("f2")
add_frame!(cache, () -> Transform3D(f2, f1, nquatrand(), rand(Vec{3, Float64})));

updating world 1
updating parent
updating world 2
updating world 1
updating parent


In [12]:
transform_to_parent(cache, f1)

Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),-0.49745978229620863 + 0.33545902760483476im + 0.47209568250761064jm - 0.6458534449500913km,FixedSizeArrays.Vec{3,Float64}((0.6870469773194985,0.7646969945415918,0.10387043519118011)))

In [13]:
transform_to_world(cache, f1)

Transform3D{Float64}(CartesianFrame3D("f1"),CartesianFrame3D("root"),-0.49745978229620863 + 0.33545902760483476im + 0.47209568250761064jm - 0.6458534449500913km,FixedSizeArrays.Vec{3,Float64}((0.6870469773194985,0.7646969945415918,0.10387043519118011)))

In [27]:
relative_transform(cache, root, f1)

updating world 1


Transform3D{Float64}(CartesianFrame3D("root"),CartesianFrame3D("f1"),-0.49745978229620863 - 0.33545902760483476im - 0.47209568250761064jm + 0.6458534449500913km,FixedSizeArrays.Vec{3,Float64}((-0.5449858088988054,0.3672331125199303,0.7973170128496235)))

In [25]:
relative_transform(cache, root, f2)

Transform3D{Float64}(CartesianFrame3D("root"),CartesianFrame3D("f2"),-0.684108046456596 - 0.28579439467503065im + 0.38839718424483616jm + 0.5472342935309845km,FixedSizeArrays.Vec{3,Float64}((-0.748361376424191,-0.4387168024447599,-0.6695410224977999)))

In [11]:
setdirty!(cache)

In [26]:
setdirty!(cache, cache.transformsToRoot[f1])

In [10]:
cache.transformsToParent[f2]

MutableCacheElement{Transform3D{Float64}}(Transform3D{Float64}(CartesianFrame3D("f2"),CartesianFrame3D("f1"),0.7012510951021995 + 0.11703332118722075im - 0.5477846196071281jm + 0.44100126289130803km,FixedSizeArrays.Vec{3,Float64}((0.23487057833918423,0.22833329687282844,0.6611491081782601))),(anonymous function),false)

In [10]:
cache.dependents[cache.dependents[cache.transformsToParent[f2]][1]]

0-element Array{CacheElement{Transform3D{Float64}},1}

In [9]:
cache.dependents[cache.transformsToRoot[f2]]

0-element Array{CacheElement{Transform3D{Float64}},1}

In [ ]:
# J = rand(Mat{3, 3, Float64}); J = J * J'
# c = rand(Vec{3, Float64})
# m = rand()
# inertia = SpatialInertia(f3, J, c, m)
# m = Mechanism()
# world = add_body!(m, RigidBody("world"))
# body = add_body!(m, RigidBody("body", inertia))
# add_joint!(m, QuaternionFloatingJoint("joint"), world, body)

In [ ]:
# transform(inertia, t3)